In [ ]:

"""
Time-resolved layered-medium reflectance
Reproduces the n=0,1,2 curves in Tualle et al., Fig. 4
-------------------------------------------------------
Dependencies: numpy, scipy, matplotlib
Runtime: a few seconds for 150 time points
"""


# -- optical properties from Fig. 4 caption ------------------------
rho     = 2.5   # mm, source–detector separation
L       = 6.0   # mm, layer-1 thickness

# layer 1
n1, mu_a1_cm, ltr1 = 1.33, 0.026, 1.0      # ltr1 in mm
mu_a1   = mu_a1_cm / 10.                   # mm⁻¹
mu_s1p  = 1.0 / ltr1                       # mm⁻¹
D1      = 1.0 / (3*(mu_a1 + mu_s1p))
c1      = 3e11 / (n1*1e3)                  # mm ns⁻¹

# layer 2 (semi-infinite)
n2, mu_a2_cm, ltr2 = 1.33, 0.05, 1/3
mu_a2   = mu_a2_cm / 10.
mu_s2p  = 1.0 / ltr2
D2      = 1.0 / (3*(mu_a2 + mu_s2p))
c2      = 3e11 / (n2*1e3)
alpha   = np.sqrt(mu_a2 / D2)

# buried isotropic source depth (≈ one transport mean-free path in layer 1)
Zsrc = ltr1

# reflection coefficient γ
gamma = (D1*np.sqrt(mu_a1/D1) - D2*np.sqrt(mu_a2/D2)) / \
        (D1*np.sqrt(mu_a1/D1) + D2*np.sqrt(mu_a2/D2))

# -- helper kernels -------------------------------------------------
def G1(z, t):
    return c1 * np.exp(-z*z / (4*D1*c1*t)) / np.sqrt(4*np.pi*D1*c1*t)

def dG1_dz(z, t):
    return -z / (2*D1*c1*t) * G1(z, t)

def g2_derivative(m, x, tau):
    # guard τ = 0 to avoid division-by-zero
    if tau == 0.0:
        return 0.0
    a = 1.0 / (4*D2*c2*tau)
    base = np.exp(-x*x*a) / np.sqrt(4*np.pi*D2*c2*tau) * np.exp(-mu_a2*c2*tau)
    return (-1)**m * (2*a)**m * hermite(m)(np.sqrt(a)*x) * base

def psi(rho, t, tau):
    num = np.exp(-rho*rho / (4*D1*c1*(t-tau) + 4*D2*c2*tau))
    den = 4*np.pi * (D1*c1*(t-tau) + D2*c2*tau)
    return num/den * np.exp(-mu_a1*c1*(t-tau) - mu_a2*c2*tau)

# -- single Iₙᵏ term ------------------------------------------------
def I_nk_surface(n, k, t):
    """2 D1 ∂/∂z term at z=0 (surface)."""
    m   = k + 2
    z_n = (-1)**n * Zsrc + 2*n*L

    def integrand(zp, tau):
        s   = -zp - z_n                 # argument of G₁ at z=0
        return psi(rho, t, tau) * zp**k * dG1_dz(s, t-tau) * \
               g2_derivative(m, alpha*zp, tau)

    # integrate τ first (0 → t), then z′ (0 → z_max)
    def tau_integral(zp):
        return quad(integrand, 0.0, t, args=(zp,), epsrel=1e-5)[0]

    z_sigma = np.sqrt(4*D2*c2*t)
    z_max   = 6*z_sigma
    return quad(tau_integral, 0.0, z_max, epsrel=1e-5)[0]

# -- reflectance vs time -------------------------------------------
def reflectance(time_ns, n_max):
    t_s  = time_ns * 1e-9
    R    = np.zeros_like(t_s)
    for n in range(n_max+1):
        weight_n = gamma**n
        for k in (0, 1):                # k_max = 1 is usually enough
            R += weight_n * np.array([I_nk_surface(n, k, t) for t in t_s])
    return 2*D1 * R       # multiply by 2 D₁ per boundary condition

# -- build curves ---------------------------------------------------
t_ns = np.linspace(0.05, 2.5, 150)      # start at 50 ps to avoid t=0
R0   = reflectance(t_ns, n_max=0)
R1   = reflectance(t_ns, n_max=1)
R2   = reflectance(t_ns, n_max=2)

# normalise by the peak of the semi-infinite curve for easy overlay
peak = R0.max()
plt.figure(figsize=(6,4))
plt.plot(t_ns, R0/peak, label='n = 0 (semi-infinite)')
plt.plot(t_ns, R1/peak, label='n → 1')
plt.plot(t_ns, R2/peak, label='n → 2')
plt.xlabel('time (ns)')
plt.ylabel('normalised reflectance')
plt.title('Time-resolved reflectance at ρ = 2.5 mm, z = 0')
plt.legend()
plt.tight_layout()
plt.show()